<a href="https://colab.research.google.com/github/cmichailidis/SurvivorPartB/blob/master/cancer_detection_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Upload your Kaggle token**

In [0]:
from google.colab import files
files.upload()

**Install the Kaggle API**

In [0]:
!pip install -q kaggle

**Copy your Kaggle token to the appropriate directory**

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

**Download the histopathologic cancer prediction dataset**

(Normally, this won't take more than three minutes to complete)

In [0]:
!kaggle competitions download -c histopathologic-cancer-detection

  0% 0.00/1.33M [00:00<?, ?B/s]
100% 1.33M/1.33M [00:00<00:00, 88.0MB/s]
 98% 5.00M/5.10M [00:00<00:00, 44.4MB/s]
100% 5.10M/5.10M [00:00<00:00, 32.5MB/s]
 99% 1.30G/1.30G [00:13<00:00, 116MB/s]
100% 1.30G/1.30G [00:13<00:00, 107MB/s]
100% 4.97G/4.98G [01:00<00:00, 96.5MB/s]
100% 4.98G/4.98G [01:00<00:00, 88.7MB/s]


**Unzip the Dataset**

(This will take approximately 5 to 10 minutes...)

In [0]:
!pwd
!ls

/content
kaggle.json  sample_submission.csv.zip	train_labels.csv.zip
sample_data  test.zip			train.zip


In [0]:
from zipfile import ZipFile

# Unzip the training dataset
zipped_training_dataset = ZipFile('train.zip', 'r')
zipped_training_dataset.extractall('train')
zipped_training_dataset.close()

# Unzip the test dataset
zipped_test_dataset = ZipFile('test.zip', 'r')
zipped_test_dataset.extractall('test')
zipped_test_dataset.close()

# Unzip the csv file which contains the labels of the dataset
!unzip train_labels.csv

Archive:  train_labels.csv.zip
  inflating: train_labels.csv        


**Split the dataset and get the training and cross-validation datasets**

In [0]:
#TODO

**Split the training dataset into minibatches**

(This has to be done because the dataset cannot fit on the GPU memory)

In [0]:
import os, numpy as np, cv2, random

def batch_generator(input_dir, labels_file, batch_size):
  with open(labels_file, 'r') as labels_file:
    labels_dict = {}
    next(labels_file)
    os.chdir(input_dir)
    
    for line in labels_file:
      image_id, label = line.split(',')
      image_id, label = image_id + '.tif', int(label)
      labels_dict[image_id] = label
      
    image_ids = np.array(labels_dict.keys())
    m = len(os.listdir(input_dir))
    N = m//batch_size
    
    if N==0:
      raise ValueError('Batch size is bigger than the dataset!')
    
    while True:
      np.random.shuffle(image_ids)
      batches = [image_ids[i*batch_size:(i+1)*batch_size] for i in range(N)] \
      + [image_ids[N*batch_size:]]
      
      for minibatch in batches:
        x_train, y_train = [], []
        
        for image in minibatch:
          x_train.append(cv2.imread(image)/256.0)
          y_train.append(labels_dict[image])
        
        yield np.array(x_train), np.array(y_train)

**Define the architecture of your prediction model**

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers import MaxPooling2D as Pool2D
from keras.layers import Convolution2D as Conv2D
from keras.layers import BatchNormalization as BatchNorm
from keras.optimizers import SGD, Adam, Nadam, RMSprop

In [0]:
#TODO
model = Sequential([
    Conv2D(filtes=4,
           kernel_size=(3,3),
           activation='relu',
           data_format='channels_last',
           input_shape=(h,w,d)),
    
    Pool2D(pool_size=(2,2)),
    
    Conv2D(filters=8,
           kernel_size=(3,3),
           activation='relu'),
    
    Pool2D(pool_size=(2,2)),
    
    Conv2D(filters=16,
           kernel_size=(3,3),
           activation='relu'),
    
    Pool2D(pool_size=(2,2)),
    
    Conv2D(filters=32,
           kernel_size=(3,3),
           activation='relu'),
    
    Pool2D(pool_size=(2,2)),
    
    Flatten(),
    Dropout(0.5),
    
    Dense(units=128,
          activation='relu'),
    
    Dropout(0.4),
    
    Dense(units=64,
          activation='relu'),
    
    Dropout(0.3),
    
    Dense(units=1,
          activation='sigmoid')   
])

In [0]:
#TODO
model.summary()

**Train the neural network**

In [0]:
#TODO
model.compile()

In [0]:
#TODO
model.fit_generator()

**Test your CNN on the cross-validation dataset to make sure that you have not overfit the dataset**

In [0]:
#TODO
model.evaluate()

**Test your CNN on the test dataset and store your predictions on a csv file**

In [0]:
#TODO
with open('output.csv') as output_file:
  predictions = model.predict(x_test)
  
  output.write('Id,Label\n')
  
  for prediction in predictions:
    output.write('{:s},{:d}\n'.format(prediction, labels_dict[]))